<a href="https://colab.research.google.com/github/BengIslam7/TinyML-ESP32-SensorPredict/blob/main/ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!unzip -q '/content/gdrive/MyDrive/28667981.zip'

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2

In [4]:
data = pd.read_csv('/content/sensor_data_2025-03-01.csv')

In [5]:
data.head()

,timestamp,temperature,humidity,light,pH,electrical_conductivity
0,2025-03-01 00:00:00,24.69,52.66,118.65,6.68,0.996
1,2025-03-01 00:00:05,21.89,50.18,801.71,7.29,1.044
2,2025-03-01 00:00:10,23.88,48.01,682.39,7.66,1.354
3,2025-03-01 00:00:15,20.42,56.73,146.05,7.77,0.519
4,2025-03-01 00:00:20,20.79,41.54,106.87,7.77,1.900


In [6]:

data[['temperature','humidity','light','pH','electrical_conductivity']].isnull().sum()

,0
temperature,0
humidity,0
light,0
pH,0
electrical_conductivity,0


# Predict next temperature values with Linear Regression

In [ ]:
temp = data['temperature'].values
real_temp = data['temperature'].values

In [ ]:
temp

array([24.69, 21.89, 23.88, ..., 21.78, 21.13, 22.9 ])

In [ ]:
temp.shape

(17280,)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
temp = scaler.fit_transform(temp.reshape(-1,1))

In [ ]:
temp

array([[0.938],
       [0.378],
       [0.776],
       ...,
       [0.356],
       [0.226],
       [0.58 ]])

In [ ]:
temp.shape

(17280, 1)

In [ ]:
X, y = [],[]
for i in range(temp.shape[0] - 10):
  X.append(temp[i:i+10])
  y.append(temp[i+10])
X=np.array(X)
y=np.array(y)
X = X.reshape(X.shape[0], -1)
print(X.shape, y.shape)

(17270, 10) (17270, 1)


In [ ]:
X

array([[0.938, 0.378, 0.776, ..., 0.82 , 0.288, 0.134],
       [0.378, 0.776, 0.084, ..., 0.288, 0.134, 0.54 ],
       [0.776, 0.084, 0.158, ..., 0.134, 0.54 , 0.044],
       ...,
       [0.38 , 0.422, 0.906, ..., 0.322, 0.264, 0.602],
       [0.422, 0.906, 0.022, ..., 0.264, 0.602, 0.356],
       [0.906, 0.022, 0.498, ..., 0.602, 0.356, 0.226]])

In [ ]:
y

array([[0.54 ],
       [0.044],
       [0.87 ],
       ...,
       [0.356],
       [0.226],
       [0.58 ]])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(1, input_shape=(10,), activation='linear')
])

model.compile(
    optimizer='adam',
    loss='mse'
)

model.fit(X, y, epochs=100)


Epoch 1/100


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


540/540 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.6057
Epoch 2/100
540/540 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1623
Epoch 3/100
540/540 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1212
Epoch 4/100
540/540 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0991
Epoch 5/100
540/540 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0902
Epoch 6/100
540/540 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0848
Epoch 7/100
540/540 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0826
Epoch 8/100
540/540 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0838
Epoch 9/100
540/540 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0845
Epoch 10/100
540/540 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0825
Epoch 11/100
540/540 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0833
Epoch 12/100
540/540 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0825
Epoch 13/100
540/540 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0822
Epoch 14/100
540/540 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0830
Epoch 15/100
540/540 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step -

In [ ]:
y_pred , y_real = [] , []
for i in range(1200,1280):
  window = temp[i:i+10].reshape(1, -1)
  prediction_scaled = model.predict(window)
  prediction = scaler.inverse_transform(prediction_scaled)
  #print('Predicted next temperature: ',prediction[0][0])
  #print('Temperature real value: ',real_temp[i+10])
  y_pred.append(prediction[0][0])
  y_real.append(real_temp[i+10])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━

In [ ]:
y_pred = np.array(y_pred)
y_real = np.array(y_real)


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae = mean_absolute_error(y_real, y_pred)
mse = mean_squared_error(y_real, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_real, y_pred)

print("\nModel Metrics:")
print("MAE :", mae)
print("MSE :", mse)
print("RMSE:", rmse)
print("R²  :", r2)



Model Metrics:
MAE : 1.2777312002182006
MSE : 2.208436862608052
RMSE: 1.4860810417363017
R²  : 0.000832699461513875


In [ ]:
import tensorflow as tf
save_path = './my_saved_model.keras'
model.save(save_path)

In [ ]:
import tensorflow as tf

# Load the Keras model first from the .keras file
loaded_keras_model = tf.keras.models.load_model('./my_saved_model.keras')

# Convert the Keras model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_keras_model)
tflite_model = converter.convert()

# Save the TFLite model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

Saved artifact at '/tmp/tmpv09gobeg'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 10), dtype=tf.float32, name='input_layer_4')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  134733497801360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134733497805200: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [ ]:
!xxd -i model.tflite > model.h

# Predict next sensors values with Random Forest

In [16]:
sdata = data[['temperature','humidity']].values

In [22]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
sdata = scaler.fit_transform(sdata.reshape(-1,2))

In [23]:
X, y = [],[]
for i in range(sdata.shape[0] - 10):
  X.append(sdata[i:i+10])
  y.append(sdata[i+10])
X=np.array(X)
y=np.array(y)
X = X.reshape(X.shape[0], -1)
print(X.shape, y.shape)

(17270, 20) (17270, 2)


In [24]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X, y)

RandomForestRegressor()

In [27]:
rdata = data[['temperature','humidity']].values.reshape(-1,2)

In [28]:
y_pred , y_real = [] , []
for i in range(1200,1280):
  window = sdata[i:i+10].reshape(1, -1)
  prediction_scaled = model.predict(window)
  prediction = scaler.inverse_transform(prediction_scaled)
  #print('Predicted next temperature: ',prediction[0][0])
  #print('Temperature real value: ',real_temp[i+10])
  y_pred.append(prediction[0])
  y_real.append(rdata[i+10])
y_pred = np.array(y_pred)
y_real = np.array(y_real)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae = mean_absolute_error(y_real, y_pred)
mse = mean_squared_error(y_real, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_real, y_pred)

print("\nModel Metrics:")
print("MAE :", mae)
print("MSE :", mse)
print("RMSE:", rmse)
print("R²  :", r2)


Model Metrics:
MAE : 35.26752567805233
MSE : 1427.4128230915162
RMSE: 37.78111728220218
R²  : -142.24280290212624


In [ ]:
data[1260:1270]

array([[2.2180e+01, 5.9600e+01, 6.8316e+02, 7.0700e+00, 9.4900e-01],
       [2.3900e+01, 5.2200e+01, 3.5716e+02, 6.1800e+00, 1.6850e+00],
       [2.4800e+01, 5.7610e+01, 4.5996e+02, 7.6000e+00, 1.9840e+00],
       [2.0590e+01, 5.2920e+01, 1.3342e+02, 6.2500e+00, 5.8600e-01],
       [2.0400e+01, 4.2530e+01, 5.0491e+02, 6.7500e+00, 1.9530e+00],
       [2.2000e+01, 4.3710e+01, 6.6329e+02, 6.3900e+00, 5.2900e-01],
       [2.0500e+01, 4.3760e+01, 6.8701e+02, 6.7800e+00, 7.8400e-01],
       [2.4050e+01, 5.9880e+01, 4.5116e+02, 6.8300e+00, 1.1930e+00],
       [2.0780e+01, 5.2360e+01, 7.4070e+02, 6.2700e+00, 7.1200e-01],
       [2.2730e+01, 4.1290e+01, 6.1761e+02, 7.4700e+00, 1.3500e+00]])

In [ ]:
data[1270]

array([ 23.54,  49.7 , 231.29,   6.76,   1.47])

In [ ]:
import joblib
joblib.dump(model, "random_forest_regressor.pkl")


['random_forest_regressor.pkl']

# Predict next sensors values with LSTM

In [7]:
sdata = data[['temperature','humidity']].values
rdata = data[['temperature','humidity']].values.reshape(-1,2)

In [8]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
sdata = scaler.fit_transform(sdata.reshape(-1,2))

In [9]:
X, y = [],[]
for i in range(sdata.shape[0] - 10):
  X.append(sdata[i:i+10])
  y.append(sdata[i+10])
X=np.array(X)
y=np.array(y)
X = X.reshape(X.shape[0], -1)
print(X.shape, y.shape)

(17270, 20) (17270, 2)


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [11]:
model = Sequential()
model.add(LSTM(units=128, return_sequences=True,
          input_shape=(X.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=128))
model.add(Dropout(0.2))
model.add(Dense(2))

model.compile(optimizer='adam', loss='mean_squared_error')

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [13]:
model.fit(X,y,epochs=10)

Epoch 1/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 28s 52ms/step - loss: 0.0839
Epoch 2/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 21s 39ms/step - loss: 0.0840
Epoch 3/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 41s 39ms/step - loss: 0.0841
Epoch 4/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 20s 37ms/step - loss: 0.0837
Epoch 5/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 23s 42ms/step - loss: 0.0837
Epoch 6/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 23s 42ms/step - loss: 0.0838
Epoch 7/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 21s 39ms/step - loss: 0.0842
Epoch 8/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 21s 38ms/step - loss: 0.0832
Epoch 9/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 20s 37ms/step - loss: 0.0845
Epoch 10/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 21s 39ms/step - loss: 0.0842


In [14]:
y_pred , y_real = [] , []
for i in range(1200,1280):
  window = sdata[i:i+10].reshape(1, -1)
  prediction_scaled = model.predict(window)
  prediction = scaler.inverse_transform(prediction_scaled)
  #print('Predicted next temperature: ',prediction[0][0])
  #print('Temperature real value: ',real_temp[i+10])
  y_pred.append(prediction[0])
  y_real.append(rdata[i+10])
y_pred = np.array(y_pred)
y_real = np.array(y_real)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae = mean_absolute_error(y_real, y_pred)
mse = mean_squared_error(y_real, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_real, y_pred)

print("\nModel Metrics:")
print("MAE :", mae)
print("MSE :", mse)
print("RMSE:", rmse)
print("R²  :", r2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━

In [ ]:
data[1260:1270]

array([[2.2180e+01, 5.9600e+01, 6.8316e+02, 7.0700e+00, 9.4900e-01],
       [2.3900e+01, 5.2200e+01, 3.5716e+02, 6.1800e+00, 1.6850e+00],
       [2.4800e+01, 5.7610e+01, 4.5996e+02, 7.6000e+00, 1.9840e+00],
       [2.0590e+01, 5.2920e+01, 1.3342e+02, 6.2500e+00, 5.8600e-01],
       [2.0400e+01, 4.2530e+01, 5.0491e+02, 6.7500e+00, 1.9530e+00],
       [2.2000e+01, 4.3710e+01, 6.6329e+02, 6.3900e+00, 5.2900e-01],
       [2.0500e+01, 4.3760e+01, 6.8701e+02, 6.7800e+00, 7.8400e-01],
       [2.4050e+01, 5.9880e+01, 4.5116e+02, 6.8300e+00, 1.1930e+00],
       [2.0780e+01, 5.2360e+01, 7.4070e+02, 6.2700e+00, 7.1200e-01],
       [2.2730e+01, 4.1290e+01, 6.1761e+02, 7.4700e+00, 1.3500e+00]])

In [ ]:
data[1270]

array([ 23.54,  49.7 , 231.29,   6.76,   1.47])